In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing the Libraries

In [ ]:
!pip install scikit-surprise
import os
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise.accuracy import rmse
from collections import defaultdict
from surprise import KNNBasic
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import accuracy
from surprise import AlgoBase
from surprise.model_selection import KFold


# Helper functions

In [ ]:
# Function to load the ratings and movies datasets

dir = '/content/drive/MyDrive/SWM/Data'
def loadDataset():
  ratings_df = pd.read_csv(dir+'/ratings.csv')
  movies_df = pd.read_csv(dir+'/movies.csv')
  
  ratings_df.drop('timestamp',axis =1, inplace = True)

  return ratings_df,movies_df


In [ ]:
# Calculates the evaluations metrics Precision and Recall for each loop of cross validation

def metricsAtK(predictions, k=10, threshold=3.5):

    # dictionary to store user and the respective predicted ratings of the movies
    user_predicted = defaultdict(list)
    for u_id, _, rating_actual, rating_predicted, _ in predictions:
        user_predicted[u_id].append((rating_predicted, rating_actual))

    precisions = dict()
    recalls = dict()

    for u_id, u_ratings in user_predicted.items():
        u_ratings.sort(key=lambda x: x[0], reverse=True)

        # Calculating actual values of the ratings
        actual = sum((ratings_true >= threshold) for (_, ratings_true) in u_ratings)
        # Calculating predicted values of the ratings
        predicted = sum((estimate_value >= threshold) for (estimate_value, _) in u_ratings[:k])

        # Calculating true positives and negatives
        positive_true = sum(((ratings_true >= threshold) and (estimate_value >= threshold))
                              for (estimate_value, ratings_true) in u_ratings[:k])
        negative_true = sum(((ratings_true < threshold) and (estimate_value < threshold))
                              for (estimate_value, ratings_true) in u_ratings[:k])
        
        # Calculating actual and predicted positives and negatives
        positive_predicted = predicted if predicted != 0 else 1
        negative_predicted = predicted if predicted != 1 else 0
        positive_actual = actual if actual != 0 else 1
        negative_actual = actual if actual != 1 else 0

        # Calculating precision, recall and accuracy
        precisions[u_id] = positive_true / positive_predicted
        recalls[u_id] = positive_true / positive_actual

    return precisions, recalls

In [ ]:
def fit(model, ratings_data, num_of_splits=5):
  # Generator object for k - cross validation
  kf = KFold(n_splits=num_of_splits)
  split_df = list()

  i = 1
  for train, test in kf.split(ratings_data):
      predictions = model.fit(train).test(test)
      rmse = accuracy.rmse(predictions, verbose=False)
      mae = accuracy.mae(predictions, verbose=False)
      precisions, recalls = metricsAtK(predictions, k=5, threshold=4)
      precision = sum(prec for prec in precisions.values()) / len(precisions)
      recall = sum(rec for rec in recalls.values()) / len(recalls)
      f1_score = (2*precision*recall)/(precision+recall)
      split_df.append([i,precision,recall,rmse,f1_score,mae])
      i +=1

  split_df = pd.DataFrame(split_df, columns=['Split', 'Precision', 'Recall', 'RMSE', 'F1 Score', 'MAE'])
  return split_df

In [ ]:
def getSortedPredictions(predictions):
    
    sorted_predictions = defaultdict(list)    
    for u_id, id, _, rating_predicted, _ in predictions:
        sorted_predictions[u_id].append((id, rating_predicted))

    for u_id, u_ratings in sorted_predictions.items():
        u_ratings.sort(key=lambda x: x[1], reverse=True)

    return sorted_predictions

In [ ]:
def inference(model):
  trainset = ratings_data.build_full_trainset()

  # build_anti_testset will generate the entires for the movies which the user has not rated. i.e 
  # The completement of the user's ratings. It assumes the rating to be equal to the global mean of the ratings.
  testset = trainset.build_anti_testset() 

  # Training the model with trainset and getting predictions using the generated testset
  model.fit(trainset)
  predictions = model.test(testset)

  return predictions


In [ ]:
# Post processing the predictions to produce the list of recommendations
def getRecommendations(predictions, n=10):

  # Getting sorted predictions
  total = getSortedPredictions(predictions)

  # Extracting only n number of movie predictions
  for user_id, user_ratings in total.items():
      total[user_id] = user_ratings[:n]

  total_df = pd.DataFrame.from_dict(total)
  total_df = total_df.transpose()

  result = []
  for user_id,user_ratings in total.items():
    result.append(total_df.loc[user_id])

  #Developing recommendations
  recommendations = []
  for i in result:
    recommended_movieIds=[]
    for x in range(0, n):
      recommended_movieIds.append(i[x][0])
    recommendations.append(recommended_movieIds)

  recommendation_list = []
  for i in recommendations:
    df = movies_df[movies_df['movieId'].isin(i)]
    temp = df['title'].tolist()
    recommendation_list.append(temp)

  recommendation_df = pd.DataFrame(recommendation_list)
  return recommendation_df


# Loading Dataset

In [ ]:
ratings_df , movies_df = loadDataset()

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))
ratings_keys = ['userId', 'movieId', 'rating']
ratings_filter = ratings_df[ratings_keys]
ratings_data = Dataset.load_from_df(ratings_filter, reader)

# Modelling Section

## K Nearest Neighbors

### Using MSD similarity

In [ ]:
msd_knn = KNNBasic(k= 40, n_epochs=20)
#msd_preds = msd_knn.fit(trainset).test(testset)


In [ ]:
# Generating evaluation metrics Precision, Recall, F1-Score, RMSE, MAE for the ratings dataset
msd_metrics = fit(msd_knn, ratings_data)
msd_metrics

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


,Split,Precision,Recall,RMSE,F1 Score,MAE
0,1,0.664180,0.263817,0.944040,0.377635,0.723022
1,2,0.674536,0.275144,0.951531,0.390857,0.729688
2,3,0.687219,0.280952,0.950258,0.398846,0.726445
3,4,0.650710,0.259220,0.947297,0.370747,0.725971
4,5,0.670744,0.265253,0.942346,0.380165,0.723221


In [ ]:
msd_predictions = inference(msd_knn)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
msd_df = getRecommendations(msd_predictions)
msd_df.to_csv(dir+'/msd_knn.csv',index = False)


### Using Pearson Correlation

In [ ]:
# Defining the similarity options with pearson correlation.
sim_options = {
    'name': 'pearson'
}

pearson_knn = KNNBasic(k= 35, n_epochs=25,sim_options = sim_options)
#cosine_preds = cosine_knn.fit(trainset).test(testset)



In [ ]:
# Generating evaluation metrics Precision, Recall, F1-Score, RMSE, MAE for the ratings dataset
pearson_metrics = fit(pearson_knn, ratings_data)
pearson_metrics

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


,Split,Precision,Recall,RMSE,F1 Score,MAE
0,1,0.660280,0.260717,0.982586,0.373826,0.759654
1,2,0.660837,0.252177,0.978962,0.365050,0.755759
2,3,0.654044,0.251259,0.973018,0.363049,0.751042
3,4,0.657814,0.257065,0.966300,0.369669,0.742633
4,5,0.643377,0.242410,0.969614,0.352141,0.749819


In [ ]:

pearson_predictions = inference(pearson_knn)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [ ]:

pearson_df = getRecommendations(pearson_predictions)
pearson_df.to_csv(dir+'/pearson_knn.csv',index = False)

### Using Cosine similarity

In [ ]:
# Defining the similarity options with cosine similarity.
sim_options = {
    'name': 'cosine'
}

cosine_knn = KNNBasic(k= 35, n_epochs=25)
#cosine_preds = cosine_knn.fit(trainset).test(testset)


In [ ]:
# Generating evaluation metrics Precision, Recall, F1-Score, RMSE, MAE for the ratings dataset
cosine_metrics = fit(cosine_knn, ratings_data)
cosine_metrics

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


,Split,Precision,Recall,RMSE,F1 Score,MAE
0,1,0.681475,0.276803,0.952263,0.393694,0.728000
1,2,0.665791,0.263455,0.945822,0.377523,0.726209
2,3,0.665546,0.267258,0.937757,0.381372,0.717676
3,4,0.661475,0.259585,0.950140,0.372851,0.728267
4,5,0.674001,0.275606,0.942943,0.391233,0.722365


In [ ]:
cosine_predictions = inference(cosine_knn)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
cosine_df = getRecommendations(cosine_predictions)
cosine_df.to_csv(dir+'/cosine_knn.csv',index = False)
